# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

output_data_file = "../output_data/cities.csv"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
got_data = pd.read_csv(output_data_file)
got_data

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,puerto ayora,39,EC,1580266934,11,-0.74,-90.35,296.48,0.89
1,1,thompson,90,CA,1580266874,92,55.74,-97.86,261.15,2.60
2,2,tasiilaq,100,GL,1580267128,86,65.61,-37.64,272.15,11.80
3,3,illapel,17,CL,1580267129,89,-31.63,-71.17,285.69,0.87
4,4,kapaa,1,US,1580266921,61,22.08,-159.32,300.15,2.60
...,...,...,...,...,...,...,...,...,...,...
545,545,corrente,100,BR,1580267829,98,-3.74,-43.34,296.51,0.83
546,546,boqueirao,40,BR,1580267830,88,-7.50,-36.13,297.15,4.10
547,547,nueva loja,31,EC,1580267831,82,0.08,-76.88,296.53,1.13
548,548,paraiso,88,CR,1580267710,96,9.84,-83.87,292.59,0.50


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


In [4]:
fig = gmaps.figure(center=(0,0), zoom_level=2)

locations = got_data[["Lat", "Lng"]].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights=got_data["Humidity"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 80 degrees but higher than 70.

# Wind speed less than 10 mph.

# Zero cloudiness.
narrowed_city_df = got_data.loc[(got_data["Cloudiness"] == 0) & (got_data["Max Temp"] < 299.817) & (got_data["Max Temp"] > 294.261) & (got_data["Wind Speed"] < 10)].dropna()
narrowed_city_df



,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
51,51,saint-georges,0,GD,1580267189,83,12.06,-61.75,299.47,6.91
59,59,geraldton,0,AU,1580266979,61,-28.77,114.60,298.15,6.70
146,146,inhambane,0,MZ,1580267308,64,-23.86,35.38,297.35,2.73
180,180,talara,0,PE,1580267351,87,-4.58,-81.27,296.65,4.38
199,199,mettur,0,IN,1580267376,78,11.80,77.80,295.12,0.73
224,224,kyaikkami,0,MM,1580267409,68,16.08,97.57,294.56,3.03
239,239,bouillante,0,GP,1580267432,83,16.13,-61.77,296.15,0.50
277,277,anastacio,0,BR,1580267483,86,-20.48,-55.81,296.60,1.69
327,327,villa carlos paz,0,AR,1580267548,73,-31.42,-64.50,298.15,4.10
354,354,piacabucu,0,BR,1580267583,75,-10.41,-36.43,299.20,3.54


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
hotel_df = pd.DataFrame({"City": [], 
                         "Country": [], 
                         "Lat": [], 
                         "Lng": []
                        })

# hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
# hotel_df["Hotel Name"] = ""


target_radius = "5000"
target_type = "lodging"

params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}


# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in narrowed_city_df.iterrows():
    params["location"] = str(row["Lat"])+ "," + str(row["Lng"])

    # run a request using our params dictionary
    try:
        response = requests.get(base_url, params=params).json()
        hotel_df = hotel_df.append({"Hotel Name": response["results"][0]["name"],
                                        "City": row["City"],
                                        "Country": row["Country"],
                                        "Lat": response["results"][0]["geometry"]["location"]["lat"],
                                        "Lng": response["results"][0]["geometry"]["location"]["lng"]}, 
                                   ignore_index=True)
        
        # Drop empty rows
        hotel_df = hotel_df.dropna()

    except(IndexError):
        print("Issue found! Go to next city... ")    
        
Hotel_Name = hotel_df["Hotel Name"]        
        
hotel_df


,City,Country,Lat,Lng,Hotel Name
0,saint-georges,GD,12.025478,-61.757204,Radisson Grenada Beach Resort
1,geraldton,AU,-28.749328,114.619364,Broadwater Mariner Resort
2,inhambane,MZ,-23.871711,35.383177,MHL Turismo SA
3,talara,PE,-4.583295,-81.272918,Hostal Oro Negro
4,mettur,IN,11.786923,77.799667,Hotel Akash Residency
5,kyaikkami,MM,16.072650,97.568357,Happy Family Kyaikkami
6,bouillante,GP,16.102095,-61.764825,Gîtes Dardanelle
7,anastacio,BR,-20.494918,-55.808491,HOTEL ARARAS
8,villa carlos paz,AR,-31.415132,-64.500932,Monte Carlo
9,piacabucu,BR,-10.407092,-36.427629,Piaçabuçu News


In [29]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel_Name'

In [ ]:
# Add marker layer ontop of heat map

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display Map